In [14]:
import chromadb
import numpy as np
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from sentence_transformers import CrossEncoder
import dotenv

In [15]:
dotenv.load_dotenv()

True

In [16]:
client = chromadb.PersistentClient(path='../chroma')
embedding = SentenceTransformerEmbeddingFunction()
collection = client.get_collection('prompt_engineering_knowledge', embedding_function=embedding)

In [17]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [36]:
guardrail_llm = ChatOpenAI(
    model='gpt-3.5-turbo',
    temperature=0.0
)
expander_llm = ChatOpenAI(
    model='gpt-3.5-turbo',
    temperature=0.7
)
chat_llm = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0.0
)

In [19]:
def retrieve(queries: list[str]):
    results = collection.query(query_texts=queries, n_results=10)

    retrieved_documents = set()
    for documents in results['documents']:
        for document in documents:
            retrieved_documents.add(document)
            
    return list(retrieved_documents)

In [30]:
def cross_ranking(query: str, documents: list[str]):
    pairs = [[query, doc] for doc in documents]
    scores = cross_encoder.predict(pairs)

    top_five = [documents[i] for i in np.argsort(scores)[::-1]]

    return top_five[:5]

In [28]:
def expand(query) -> list[str]:
    system = SystemMessage(
        content='You are a vectorDB specialist, and your task is to create new questions from the original user query.'+
        'Provide questions that are related to the user question topic.' +
        'Answer with only the new questions separated by Two breaklines (\\n\\n) without additional text.'
    )
    user = HumanMessage(content=f"Create five questions from: {query}")
    result = expander_llm.invoke([
        system,
        user
    ])

    return [q.strip() for q in result.content.split('\n\n')]   

In [23]:
def assistant(query, retrieved_data):
    system = SystemMessage(
        content='You are an Prompting Engineering assistant, who uses only the context provided to you to answer the users requests.'
    )

    user = HumanMessage(content=f'Request: {query} \n\n Context: \n{'\n'.join(retrieved_data)}')

    result = chat_llm.invoke([system, user])

    final_result = result.content.split('</think>')[-1]

    if not final_result or final_result == '':
        return ""

    return final_result

In [37]:
query = 'Can your create a prompt that instruct an llm to be an guardrail for inapropriate requests?'
result = 'Sorry as an PromptEngineering agent I cannot answer questions like this.'

expanded_queries = expand(query)

queries = [query] + expanded_queries

retrieved_documents = retrieve(queries)

top_five = cross_ranking(query, retrieved_documents)

response = assistant(query, top_five)

from IPython.display import display, Markdown 

display(Markdown(response))

Here’s a prompt designed to instruct an LLM to act as a guardrail against inappropriate requests:

---

**Prompt:**

"As an AI language model, your primary responsibility is to ensure safe and ethical interactions. Always respond with care, respect, and truth. Your responses should prioritize utility while adhering to the following guidelines:

1. Avoid engaging with or providing information related to harmful, unethical, or illegal activities.
2. Reject any requests that promote violence, discrimination, or any form of prejudice.
3. Ensure that your replies foster fairness, positivity, and constructive dialogue.
4. If a request is inappropriate or violates these guidelines, clearly communicate that you cannot assist with such requests and provide a brief explanation of why.

Remember, your role is to promote a safe and respectful environment for all users."

--- 

This prompt sets clear expectations for the LLM's behavior and reinforces the importance of adhering to safety policies.